In [ ]:
from datawand.parametrization import ConfigGenerator

# Initialize configuration generator

In [ ]:
cg = ConfigGenerator(pipeline_name='LNGraph')

# Setting default parameters for pipeline

In [ ]:
DEFAULTS = {}

#### Set your default parameters below:

In [ ]:
DEFAULTS["data_dir"] = "/mnt/idms/fberes/data/bitcoin_ln_research"
DEFAULTS["weight"] = None
DEFAULTS["weight_cols"] = [None, "num_channels", "capacity", "fee_base_msat", "fee_rate_milli_msat"]
DEFAULTS["snapshot_ids"] = list(range(9))

In [ ]:
DEFAULTS["experiment_id"] = "trial"
DEFAULTS["snapshot_id"] = 0
DEFAULTS["num_transactions"] = 7000
DEFAULTS["amount_sat"] = 60000
DEFAULTS["eps"] = 0.8 
DEFAULTS["day_interval"] = 7
DEFAULTS["drop_disabled"] = True
DEFAULTS["drop_low_cap"] = True
DEFAULTS["with_depletion"] = True

In [ ]:
DEFAULTS["sim_root_dir"] = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_7days/"
DEFAULTS["sim_res_dir"] = "2019-09-25_15:48:42_50000sat_k6000_eps0.00"
DEFAULTS["snapshots"] = list(range(7))

# Setting parameters for notebooks

In [ ]:
PARAMETERS = {}

In [ ]:
for nb_name in cg.pythonitem_names:
    PARAMETERS[nb_name] = []
print(PARAMETERS)

#### Set your parameters for each notebooks below: 

##### You can specify the ending of the names of the generated notebook clones, if you add an "\_\_ ending \_\_" parameter. (Otherwise they will end with \_CLONE\_1, \_CLONE\_2, etc.)
##### WARNING: if the name of the generated notebook clone is not an identifier, then luigi will NOT be able to run it.

## Centrality computation

for w in DEFAULTS["weight_cols"]:
    PARAMETERS["CentralityCalculator"].append({"weight":w})

## a.) Transaction simulation

In [ ]:
import datetime as dt

### i.) Transaction failure experiments

start_time = dt.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
num_samples = 5
tx_num = 1000#50000#1000#2000#10000#20000#5000

for sat in [10000,25000,50000,100000,200000,500000]:
    drop_disabled = True
    for amount, num_transactions in [(sat,tx_num)]:
        for snap in range(7):
            for i in range(num_samples):
                PARAMETERS['RunSimulator'].append({"drop_low_cap":False, "snapshot_id":snap, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%s_%isat_k%i_keep_low_cap.%i" % (start_time, amount, num_transactions, i)})

### ii.) LNBIG tuning weekly

start_time = dt.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
num_samples = 10

for sat in [50000,60000]:#[10000,25000,50000,100000,200000,500000]:
    drop_disabled = True
    for amount, num_transactions in [(sat,6500),(sat,7000)]:
        for snap in range(7):
            for i in range(num_samples):
                PARAMETERS['RunSimulator'].append({"day_interval":7, "snapshot_id":snap, "drop_disabled":drop_disabled, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%s_%isat_k%i.%i" % (start_time, amount, num_transactions, i)})

### iii.) LNBIG tuning daily

start_time = dt.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
num_samples = 10
#for amount, num_transactions in [(60000,6500)]:
for amount, num_transactions in [(10000,6500)]:
    for snap in range(40):
        for i in range(num_samples):
            PARAMETERS['RunSimulator'].append({"day_interval":1, "snapshot_id":snap, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%s_%isat_k%i.%i" % (start_time, amount, num_transactions, i)})

num_samples = 10
num_transactions = 70000
amount = 500000#200000#100000#60000#10000
for snap in range(40):
    for i in range(num_samples):
        PARAMETERS['RunSimulator'].append({"day_interval":1, "snapshot_id":snap, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%isat_k%i_eps0.80.%i" % (amount, num_transactions, i)})

### iv.) Epsilon

start_time = dt.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
num_samples = 10

amount = 50000
num_transactions = 6000
for eps in [0.0,0.2,0.5,0.8,1.0]:
    for snap in range(7):
        for i in range(num_samples):
            PARAMETERS['RunSimulator'].append({"day_interval":7, "eps":eps, "snapshot_id":snap, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%s_%isat_k%i_eps%.2f.%i" % (start_time, amount, num_transactions, eps, i)})

### v.) MULTI EXPERIMENTS

for tx_num in [1000,7000,10000,20000,50000,70000]:
    for tx_amount in [10000,20000,60000,100000,200000,500000]:
        PARAMETERS['RunSimulator1Days'].append({"eps":0.8, "amount_sat":tx_amount, "num_transactions":tx_num})

PARAMETERS['RunSimulator1Days'].append({"eps":0.0, "amount_sat":60000, "num_transactions":7000})
PARAMETERS['RunSimulator1Days'].append({"eps":0.2, "amount_sat":60000, "num_transactions":7000})
PARAMETERS['RunSimulator1Days'].append({"eps":0.5, "amount_sat":60000, "num_transactions":7000})
PARAMETERS['RunSimulator1Days'].append({"eps":1.0, "amount_sat":60000, "num_transactions":7000})

### vi.) Fee optimalization

num_samples = 10
num_transactions = 7000
amount = 60000
for with_dep in [False,True]:
    for snap in range(7):
        for i in range(num_samples):
            PARAMETERS['RunSimulator'].append({"with_depletion":with_dep, "day_interval":7, "snapshot_id":snap, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%isat_k%i_eps0.80_wdep%s.%i" % (amount, num_transactions, with_dep, i)})

In [ ]:
num_samples = 10
num_transactions = 7000
amount = 60000
for snap in range(40):
    for i in range(num_samples):
        PARAMETERS['RunSimulator'].append({"day_interval":1, "snapshot_id":snap, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%isat_k%i_eps0.80.%i" % (amount, num_transactions, i)})

### vii.) Entity failure ratio

num_samples = 10
num_transactions = 7000
amount = 60000
for snap in range(40):
    for i in range(num_samples):
        PARAMETERS['RunSimulatorWithEntityExclusion'].append({"day_interval":1, "snapshot_id":snap, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%isat_k%i_eps0.80.%i" % (amount, num_transactions, i)})

## b.) Simulation result analysis

simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_1days/"
snapshots = list(range(40))
experiments = [
    "2019-09-06_22:03:19_50000sat_k6000",
    "2019-09-07_05:06:14_25000sat_k6000",
    "2019-09-07_10:23:25_10000sat_k6000",
    "2019-09-07_14:24:11_100000sat_k6000",
    "2019-09-07_18:37:32_200000sat_k6000",
    "2019-09-07_22:40:54_500000sat_k6000"
]

for exp_id in experiments:
    PARAMETERS['NodeRoutingIncomeStats'].append({"sim_root_dir":simulation_dir, "snapshots":snapshots, "sim_res_dir":exp_id})

simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_7days/"
snapshots = list(range(7))
experiments = [
    #"2019-09-26_14:15:21_50000sat_k6000_eps0.00",
    #"2019-09-26_14:15:21_50000sat_k6000_eps0.20",
    #"2019-09-26_14:15:21_50000sat_k6000_eps0.50",
    #"2019-09-26_14:15:21_50000sat_k6000_eps0.80",
    #"2019-09-26_14:15:21_50000sat_k6000_eps1.00",
    "2019-09-27_12:50:21_50000sat_k6500",
    "2019-09-27_12:50:21_50000sat_k7000",
    "2019-09-27_12:50:21_60000sat_k6500",
    "2019-09-27_12:50:21_60000sat_k7000"
]



for exp_id in experiments:
    PARAMETERS['NodeRoutingIncomeStats'].append({"sim_root_dir":simulation_dir, "snapshots":snapshots, "sim_res_dir":exp_id})
    #PARAMETERS['SimulationStability'].append({"sim_root_dir":simulation_dir, "snapshots":snapshots, "sim_res_dir":exp_id})

simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_1days/"
snapshots = list(range(40))
experiments = [
    "2019-09-27_22:37:13_60000sat_k6500",
    "2019-09-28_07:03:52_10000sat_k6500"
]

for exp_id in experiments:
    PARAMETERS['NodeRoutingIncomeStats'].append({"sim_root_dir":simulation_dir, "snapshots":snapshots, "sim_res_dir":exp_id})
    PARAMETERS['SimulationStability'].append({"sim_root_dir":simulation_dir, "snapshots":snapshots, "sim_res_dir":exp_id})

simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_multi/"
snapshots = list(range(40))

income_experiments = [
    "10000sat_k7000_eps0.80",
    "20000sat_k7000_eps0.80",
    "60000sat_k7000_eps0.80",
    "100000sat_k7000_eps0.80",
    "200000sat_k7000_eps0.80",
    "500000sat_k7000_eps0.80",
]

for exp_id in income_experiments:
    PARAMETERS['NodeRoutingIncomeStats'].append({"sim_root_dir":simulation_dir, "snapshots":snapshots, "sim_res_dir":exp_id})

epsilon_experiments = [
    "60000sat_k7000_eps0.00",
    "60000sat_k7000_eps0.20",
    "60000sat_k7000_eps0.50",
    "60000sat_k7000_eps0.80",
    "60000sat_k7000_eps1.00",
]
    
for exp_id in epsilon_experiments:
    PARAMETERS['NodeRoutingIncomeStats'].append({"sim_root_dir":simulation_dir, "snapshots":snapshots, "sim_res_dir":exp_id})
    PARAMETERS['SimulationStability'].append({"sim_root_dir":simulation_dir, "snapshots":snapshots, "sim_res_dir":exp_id})

# Export parameters into pipeline configuration file

In [ ]:
cg.save_params(PARAMETERS, DEFAULTS)

cg.pipeline_cfg